Final Project for Tim Wu

Step one is to start h2o, load your chosen data set(s) and follow the project-specific data manipulation steps.

This is a regression problem. The model will predict "price". The goal is reach the target RMSE below 123,000

In [1]:
## Import Libraries

import h2o
from h2o.grid.grid_search import H2OGridSearch
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators.random_forest import H2ORandomForestEstimator
from h2o.estimators.deeplearning import H2ODeepLearningEstimator
from h2o.estimators.stackedensemble import H2OStackedEnsembleEstimator
import warnings
warnings.filterwarnings("ignore")
SEED = 123

In [2]:
#start h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_102"; Java(TM) SE Runtime Environment (build 1.8.0_102-b14); Java HotSpot(TM) 64-Bit Server VM (build 25.102-b14, mixed mode)
  Starting server from /Users/tim.wu/anaconda3/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/61/sfyb5b517dd3tj1_cdjsx6_w0000gn/T/tmpvvuzog6x
  JVM stdout: /var/folders/61/sfyb5b517dd3tj1_cdjsx6_w0000gn/T/tmpvvuzog6x/h2o_tim_wu_started_from_python.out
  JVM stderr: /var/folders/61/sfyb5b517dd3tj1_cdjsx6_w0000gn/T/tmpvvuzog6x/h2o_tim_wu_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.0.3
H2O cluster version age:,16 days
H2O cluster name:,H2O_from_python_tim_wu_h1kfbv
H2O cluster total nodes:,1
H2O cluster free memory:,3.556 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


In [3]:
# load your chosen data set(s)
df = h2o.import_file("http://coursera.h2o.ai/house_data.3487.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [4]:
print(df.shape)

(21613, 21)


Split date into year and month columns. Then combine them into a numeric date column

In [5]:
df["date"] = df["date"].gsub(pattern = "T000000", replacement = "")
df["date"] = df["date"].trim()

In [6]:
df_month_date = df["date"].strsplit(pattern = "20\d\d")
df_month = df_month_date[1].strsplit(pattern = "\d\d$")
df_month.set_names(['month'])

month
10
12
02
12
02
05
06
01
04
03


In [7]:
df_year = df["date"].strsplit(pattern = "\d\d\d\d$")
df_year.set_names(['year'])

year
2014
2014
2015
2014
2015
2014
2014
2015
2015
2015


In [8]:
df = df.cbind(df_year)
df = df.cbind(df_month)

In [9]:
df["year"] = df["year"].asnumeric()
df["month"] = df["month"].asnumeric()
df["date"] = df["date"].asnumeric()

In [10]:
df.summary()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,year,month
type,int,int,int,int,real,int,int,real,int,int,int,int,int,int,int,int,int,real,real,int,int,int,int
mins,1000102.0,20140502.0,75000.0,0.0,0.0,290.0,520.0,1.0,0.0,0.0,1.0,1.0,290.0,0.0,1900.0,0.0,98001.0,47.1559,-122.519,399.0,651.0,2014.0,1.0
mean,4580301520.864987,20143902.6682551,540088.1417665284,3.370841623097218,2.114757321982139,2079.899736269819,15106.96756581695,1.4943089807060526,0.007541757275713691,0.23430342849211097,3.4094295100171164,7.6568731781798105,1788.3906907879518,291.50904548188555,1971.0051357979064,84.4022579003377,98077.93980474674,47.56005251931665,-122.21389640494158,1986.5524915560036,12768.45565169118,2014.3229537778102,6.574422801091883
maxs,9900000190.0,20150527.0,7700000.0,33.0,8.0,13540.0,1651359.0,3.5,1.0,4.0,5.0,13.0,9410.0,4820.0,2015.0,2015.0,98199.0,47.7776,-121.315,6210.0,871200.0,2015.0,12.0
sigma,2876565571.3120522,4436.582468634118,367127.19648270035,0.930061831147451,0.7701631572177408,918.4408970468095,41420.51151513551,0.5399888951423489,0.08651719772788766,0.7663175692736117,0.6507430463662044,1.1754587569743344,828.0909776519175,442.57504267746685,29.373410802386235,401.67924001917555,53.50502625747248,0.13856371024192365,0.14082834238139275,685.3913042527788,27304.179631338524,0.4676160310451536,3.1153077787263648
zeros,0,0,0,13,10,0,0,0,21450,19489,0,0,0,13126,0,20699,0,0,0,0,0,0,0
missing,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,7129300520.0,20141013.0,221900.0,3.0,1.0,1180.0,5650.0,1.0,0.0,0.0,3.0,7.0,1180.0,0.0,1955.0,0.0,98178.0,47.5112,-122.257,1340.0,5650.0,2014.0,10.0
1,6414100192.0,20141209.0,538000.0,3.0,2.25,2570.0,7242.0,2.0,0.0,0.0,3.0,7.0,2170.0,400.0,1951.0,1991.0,98125.0,47.721,-122.319,1690.0,7639.0,2014.0,12.0
2,5631500400.0,20150225.0,180000.0,2.0,1.0,770.0,10000.0,1.0,0.0,0.0,3.0,6.0,770.0,0.0,1933.0,0.0,98028.0,47.7379,-122.233,2720.0,8062.0,2015.0,2.0


Split the data into train and test, using 0.9 for the ratio, and a seed of 123. That should give 19,462 training rows and 2,151 test rows.

In [11]:
# Split the train dataset
train, test = df.split_frame(ratios=[0.9], seed=SEED)

In [12]:
print(train.shape)
print(test.shape)

(19462, 23)
(2151, 23)


In [13]:
train, valid = df.split_frame(ratios=[0.8], seed=SEED)

In [14]:
# Seperate the target data and store it into y variable
y = 'price'

In [15]:
# remove target and Id column from the dataset and store rest of the columns in X variable
X = list(train.columns)
X.remove(y)
X.remove('id')

At the end of this step you will have `train`, `test`, `x` and `y` variables, and possibly `valid` also. Check you have the correct number of rows and columns (as specified in the project description) before moving on.

In [16]:
### 1. Gradient Boosting Machine (GBM)

mGBM = H2OGradientBoostingEstimator(model_id = "GBM_model_baseline",
                                   stopping_rounds = 4,
                                   stopping_metric ="RMSE",
                                   stopping_tolerance = 0.001,
                                   ntrees = 200,
                                   max_depth =5,
                                   nfolds = 5,
                                   fold_assignment = "Modulo",
                                   keep_cross_validation_predictions = True,
                                   seed = SEED)
mGBM.train(X, y, train, validation_frame = valid)
 
h2o.save_model(mGBM, "./models/")

gbm Model Build progress: |███████████████████████████████████████████████| 100%


'/Users/tim.wu/Documents/Programming_Code/python_code/python_h2o/models/GBM_model_baseline'

In [17]:
print(mGBM.model_performance(test))


ModelMetricsRegression: gbm
** Reported on test data. **

MSE: 15718405002.153336
RMSE: 125373.06330369908
MAE: 67877.76993399864
RMSLE: 0.16918213800316087
Mean Residual Deviance: 15718405002.153336



The baseline_gbm_model gives the RMSE: 125,373 which is higher than the target RMSE 123,000

In [18]:
### 2. Random Forest Algorithm

mRF = H2ORandomForestEstimator(model_id = "RF_model_baseline",
                               nfolds = 5,
                               fold_assignment = "Modulo",
                               keep_cross_validation_predictions = True,
                               seed = SEED
                              )
mRF.train(X, y, train, validation_frame = valid)
 
h2o.save_model(mRF, "./models/")

drf Model Build progress: |███████████████████████████████████████████████| 100%


'/Users/tim.wu/Documents/Programming_Code/python_code/python_h2o/models/RF_model_baseline'

In [19]:
print(mRF.model_performance(test))


ModelMetricsRegression: drf
** Reported on test data. **

MSE: 16639842953.874172
RMSE: 128995.5152471363
MAE: 70619.13117692334
RMSLE: 0.17580020448537878
Mean Residual Deviance: 16639842953.874172



The baseline_rf_model gives the RMSE: 128,995 which is higher than the target RMSE 123,000 and wrose than GBM baseline

In [20]:
from h2o.estimators.deeplearning import H2ODeepLearningEstimator
mDL = H2ODeepLearningEstimator(model_id = "DeepLearning_model_baseline",
                               nfolds = 5,
                               fold_assignment = "Modulo",
                               keep_cross_validation_predictions = True,
                               seed = SEED
                              )
mDL.train(X, y, train, validation_frame = valid)
 
h2o.save_model(mDL, "./models/")

deeplearning Model Build progress: |██████████████████████████████████████| 100%


'/Users/tim.wu/Documents/Programming_Code/python_code/python_h2o/models/DeepLearning_model_baseline'

In [21]:
print(mDL.model_performance(test))


ModelMetricsRegression: deeplearning
** Reported on test data. **

MSE: 16278741228.201817
RMSE: 127588.17040855244
MAE: 72466.37000801423
RMSLE: 0.1793398304170479
Mean Residual Deviance: 16278741228.201817



The baseline_dl_model gives the RMSE: 127,588 which is higher than the target RMSE 123,000 and wrose than GBM baseline

Step three is to train a stacked ensemble of the models I made in step two.

In [23]:
models = [mRF.model_id, mGBM.model_id, mDL.model_id]
 
mSE = H2OStackedEnsembleEstimator(model_id = "SE_model",
                                 base_models=models)
mSE.train(X, y, train, validation_frame = valid)
 
h2o.save_model(mSE, "./models/")

stackedensemble Model Build progress: |███████████████████████████████████| 100%


'/Users/tim.wu/Documents/Programming_Code/python_code/python_h2o/models/SE_model'

In [24]:
print(mSE.model_performance(test))


ModelMetricsRegressionGLM: stackedensemble
** Reported on test data. **

MSE: 114389963025.48634
RMSE: 338215.8527116763
MAE: 220449.11152929967
RMSLE: 0.5146760044641618
R^2: 0.13947267796105356
Mean Residual Deviance: 114389963025.48634
Null degrees of freedom: 2150
Residual degrees of freedom: 2147
Null deviance: 285936137785399.56
Residual deviance: 246052810467821.12
AIC: 60884.92667796433



The stackedensemble gives the RMSE: 338,215 which is higher than the target RMSE 123,000 and wrose than GBM baseline

Repeat steps two and three until your best model (which is usually your ensemble model, but does not have to be) has the minimum required performance on the validation data.

In [25]:
### Gradient Boosting Machine (GBM) after grid search

mGBM_best = H2OGradientBoostingEstimator(model_id = "GBM_model_best_4",
                                   score_tree_interval=5,     # For early stopping
                                   stopping_rounds=3,         # For early stopping
                                   stopping_tolerance=0.0005,
                                   ntrees=200,
                                   col_sample_rate = 0.2,
                                   learn_rate = 0.1,
                                   sample_rate = 0.8,
                                   seed = SEED)
mGBM_best.train(X, y,
                training_frame=train,
                validation_frame=valid)

h2o.save_model(mGBM_best, "./models/")


gbm Model Build progress: |███████████████████████████████████████████████| 100%


'/Users/tim.wu/Documents/Programming_Code/python_code/python_h2o/models/GBM_model_best_4'

Step four is to get the performance on the test data of the chosen model (GBM), and confirm that this also reaches the minimum target on the test data. Record your model performance in comments at the end of your script

In [26]:
print(mGBM_best.model_performance(test))


ModelMetricsRegression: gbm
** Reported on test data. **

MSE: 14197863207.460981
RMSE: 119154.78675848898
MAE: 69866.26518889358
RMSLE: 0.1738943780516706
Mean Residual Deviance: 14197863207.460981



After grid search, best_gbm_model gives the best RMSE: 119,155 which is lower than the target RMSE below 123,000

In [27]:
h2o.cluster().shutdown()

H2O session _sid_8298 closed.
